# Sistemas Operacionais
## Laboratório 02 - Threads e comunicação entre processos

### **Aluno:** Paulo Roberto Fernandes Holanda


**Exercício 1**

Modifique o código acima para fazer com que dois processos filhos alterem uma memória compartilhada contendo um inteiro. Faça o processo pai esperar por seus dois filhos para depois imprimir o valor da variável na tela. Dica: olhe o código do exercício 3 do Laboratório 1.


---

In [1]:
!mkdir -p build
!mkdir -p exercises

In [ ]:
%%writefile exercises/exercise1.c
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <unistd.h>
#include <sys/shm.h>
#include <sys/stat.h>
#include <stdlib.h>

int main()
{
    int segment_id = shmget(IPC_PRIVATE, sizeof(int), S_IRUSR | S_IWUSR);
    int *sharedMemory = (int *)shmat(segment_id, NULL, 0);
    *sharedMemory = 0; 
    printf("Pai com PID %d acessa a variável compartilhada com valor inicial: %d\n", getpid(), *sharedMemory);
    
    pid_t pid1 = fork();
    if (pid1 < 0) {
        printf("Erro ao criar o primeiro processo filho.\n");
        return 1;
    } else if (pid1 == 0) {  
        sharedMemory = (int *)shmat(segment_id, NULL, 0); 
        *sharedMemory += 10; 
        printf("Filho 1 com PID: %d incrementou o valor da variável em 10 para: %d\n", getpid(), *sharedMemory);
        shmdt(sharedMemory); 
        exit(0); 
    }

    pid_t pid2 = fork();
    if (pid2 < 0) {
        printf("Erro ao criar o segundo processo filho.\n");
        return 1;
    } else if (pid2 == 0) {  
        sharedMemory = (int *)shmat(segment_id, NULL, 0);  
        *sharedMemory += 10;  
        printf("Filho 2 com PID: %d incrementou o valor da variável em 10 para: %d\n", getpid(), *sharedMemory);
        shmdt(sharedMemory);  
        exit(0);
    }
    wait(NULL);
    wait(NULL);

    sharedMemory = (int *)shmat(segment_id, NULL, 0);
    printf("Pai com PID %d acessa a variável compartilhada com valor final: %d\n", getpid(), *sharedMemory);
    shmctl(segment_id, IPC_RMID, NULL);
    return 0;
}


In [ ]:
!gcc ./exercises/exercise1.c -o ./build/exercise1
!./build/exercise1




**Exercício 2**

Utilizando comunicação entre processos, crie um código que escreva a Sequência de Fibonacci, com o parâmetro n sendo uma variável global. Tal sequência deve ser gerada por um processo filho, mas deve ser impressa na tela pelo processo pai. Dica: olhe o código do desafio do Laboratório 1.


---



In [ ]:
%%writefile exercises/exercise2.c
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <unistd.h>
#include <stdlib.h>

int n = 20;
int fibonacci_n(int fib_n_1, int fib_n_2) {
    return fib_n_1 + fib_n_2;
}
int main() {
    int pipe_fd[2];
    
    if (pipe(pipe_fd) == -1) {
        perror("Erro ao criar pipe");
        return 1;
    }
    printf("Sou o processo pai com PID %d, vou criar um filho para calcular a sequência de Fibonacci.\n", getpid());
    pid_t pid = fork(); 

    if (pid < 0) {
        perror("Erro ao criar o processo filho");
        return 1;
    } else if (pid == 0) {
        printf("Sou o processo filho com PID %d, vou calcular a sequência de Fibonacci de tamanho n=%d.\n", getpid(), n);
        close(pipe_fd[0]);

        int fib[n+1];
        fib[0] = 0;
        if (n > 0) {
            fib[1] = 1;
        }

        for (int i = 2; i <= n; i++) {
            fib[i] = fibonacci_n(fib[i-1], fib[i-2]);
        }
        write(pipe_fd[1], fib, (n+1) * sizeof(int));
        close(pipe_fd[1]);
        exit(0);
    } else {
        close(pipe_fd[1]);
        wait(NULL);
        int fib[n+1];
        read(pipe_fd[0], fib, (n+1) * sizeof(int));
        close(pipe_fd[0]);

        printf("Sou o processo pai com PID %d, e essa é a sequência de Fibonacci:\n", getpid());
        for (int i = 0; i <= n; i++) {
            printf("%d ", fib[i]);
        }
        printf("\n");
    }
    return 0;
}


In [ ]:
!gcc ./exercises/exercise2.c -o ./build/exercise2
!./build/exercise2




**Exercício 3**

Usando comunicação entre processos, escreva um código que realiza a soma de *n* números naturais, sendo *n* uma variável global. Cada processo deve fazer apenas a soma de dois números e o resultado final deve ser retornado pelo primeiro processo. Por exemplo, o processo 1 deve fazer a soma de 0+1, o processo 2 vai pegar esse resultado e somar com 2, e assim por diante.


---

In [ ]:
%%writefile exercises/exercise3.c
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <unistd.h>
#include <sys/shm.h>
#include <sys/stat.h>
#include <stdlib.h>


int n = 15;
int main()
{
    int segment_id = shmget(IPC_PRIVATE, sizeof(int), S_IRUSR | S_IWUSR);
    int *sharedMemory = (int *)shmat(segment_id, NULL, 0);
    *sharedMemory = 0;
    shmdt(sharedMemory);
    
    for (int i = 1; i < n; i++) {
        pid_t pid = fork();

        if (pid < 0) {
              printf("Sou o processo pai %d e nao consegui criar um novo processo.\n",getpid());
            exit(1);
        } else if (pid == 0) {  
            sharedMemory = (int *)shmat(segment_id, NULL, 0);
            *sharedMemory += i;
            printf("Sou o processo filho %d e somei %d ao valor. O valor atual é: %d\n", getpid(), i, *sharedMemory);
            shmdt(sharedMemory);
            exit(0);
        } else {
            wait(NULL);
        }
    }
    sharedMemory = (int *)shmat(segment_id, NULL, 0);
    printf("Sou o processo pai %d e O valor final da soma é: %d\n",getpid(),*sharedMemory);
    shmctl(segment_id, IPC_RMID, NULL);
    return 0;
}


In [ ]:
!gcc ./exercises/exercise3.c -o ./build/exercise3
!./build/exercise3

**Exercício 5**

Refaça o exercício 2 usando threads.

---

In [ ]:
%%writefile exercises/exercise5.c
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <unistd.h>
#include <stdlib.h>
#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
int n = 20;  
int *fib;      

void *calculate_fibonacci(void *arg) {
    printf("sou a thread fazendo o Fibonnaci\n");
    if (n >= 1) {
        fib[0] = 0; 
    }
    if (n >= 2) {
        fib[1] = 1; 
    }

    for (int i = 2; i <= n; i++) {
        fib[i] = fib[i - 1] + fib[i - 2]; 
    }

    pthread_exit(NULL);
}

int main() {
    pthread_t thread_id;
    fib = malloc((n + 1) * sizeof(int));
    printf("Sou o processo pai com PID %d, e estou criando a thread\n", getpid());
    if (pthread_create(&thread_id, NULL, calculate_fibonacci, NULL) != 0) {
        printf("Erro ao criar a thread\n");
        free(fib); 
        return 1;
    }
    pthread_join(thread_id, NULL);

    printf("Sou o processo pai com PID %d, e essa é a sequência de Fibonacci:\n", getpid());
    for (int i = 0; i <= n; i++) {
        printf("%d ", fib[i]);
    }
    printf("\n");

    free(fib);
    return 0;
}

In [ ]:
!gcc ./exercises/exercise5.c -o build/exercise5
!./build/exercise5